imports of the program

In [ ]:
import azure.cosmos.documents as documents
import azure.cosmos.cosmos_client as cosmos_client
import azure.cosmos.exceptions as exceptions
from azure.cosmos.partition_key import PartitionKey
from datetime import datetime
import requests, os

In [ ]:
settings = {
    'host': os.environ.get('ACCOUNT_HOST', 'https://andresdia02.documents.azure.com:443/'),
    'master_key': os.environ.get('ACCOUNT_KEY', ''),
    'database_id': os.environ.get('COSMOS_DATABASE', ''),
    'container_id': os.environ.get('COSMOS_CONTAINER', '')
}

#Asignación de valores a variables internas
HOST = settings['host']
MASTER_KEY = settings['master_key']
DATABASE_ID = settings['database_id']
CONTAINER_ID = settings['container_id']

In [ ]:
def create_items(container):
    print('\nCreating Items\n')

    tonData = secDataFormat("ip1")
    container.create_item(body=tonData)
    tonData = secDataFormat("ip2")
    container.create_item(body=tonData)

def secDataFormat(item_id):
    headers = {'X-Key':'fd2c4031-0399-42f6-9c21-00b429348976'}
    #data = requests.get("https://v1.cveapi.com/CVE-2019-9956.json")
    data = requests.get("https://api.binaryedge.io/v2/query/image/ip/10.157.11.119",headers=headers)
    consume = data.json()
    dataS = {'id' : item_id,
            'partitionKey' : 'Account1',
            'date' : datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'VulnerabilityDesc' : consume
            }

    return dataS

def delete_item(container, doc_id, account_number):
    print('\nDeleting Item by Id\n')

    response = container.delete_item(item=doc_id, partition_key=account_number)

    print('Deleted item\'s Id is {0}'.format(doc_id))

def read_item(container, doc_id, account_number):
    print('\nReading Item by Id\n')

    # We can do an efficient point read lookup on partition key and id
    response = container.read_item(item=doc_id, partition_key=account_number)

    print('Item read by Id {0}'.format(doc_id))
    print('Partition Key: {0}'.format(response.get('partitionKey')))
    print('Description: {0}'.format(response.get('VulnerabilityDesc')))
    
def read_items(container):
    print('\nReading all items in a container\n')

    item_list = list(container.read_all_items(max_item_count=10))

    print('Found {0} items'.format(item_list.__len__()))

    for doc in item_list:
        print('Item Id: {0}'.format(doc.get('id')))
        
def query_items(container, account_number):
    print('\nQuerying for an  Item by Partition Key\n')

    # Including the partition key value of account_number in the WHERE filter results in a more efficient query
    items = list(container.query_items(
        query="SELECT * FROM r WHERE r.partitionKey=@account_number",
        parameters=[
            { "name":"@account_number", "value": account_number }
        ]
    ))

    print('Item queried by Partition Key {0}'.format(items[0].get("id")))

def run_sample():
    client = cosmos_client.CosmosClient(HOST, {'masterKey': MASTER_KEY}, user_agent="CosmosDBPythonQuickstart", user_agent_overwrite=True)
    try:
        # setup database for this sample
        try:
            db = client.create_database(id=DATABASE_ID)
            print('Database with id \'{0}\' created'.format(DATABASE_ID))

        except exceptions.CosmosResourceExistsError:
            db = client.get_database_client(DATABASE_ID)
            print('Database with id \'{0}\' was found'.format(DATABASE_ID))

        # setup container for this sample
        try:
            container = db.create_container(id=CONTAINER_ID, partition_key=PartitionKey(path='/partitionKey'))
            print('Container with id \'{0}\' created'.format(CONTAINER_ID))

        except exceptions.CosmosResourceExistsError:
            container = db.get_container_client(CONTAINER_ID)
            print('Container with id \'{0}\' was found'.format(CONTAINER_ID))

    except exceptions.CosmosHttpResponseError as e:
        print('\nrun_sample has caught an error. {0}'.format(e.message))

    finally:
            print("\nrun_sample done")


if __name__ == '__main__':
    client = cosmos_client.CosmosClient(HOST, {'masterKey': MASTER_KEY}, user_agent="CosmosDBPythonQuickstart", user_agent_overwrite=True)
    try:
        # setup database for this sample
        try:
            db = client.create_database(id=DATABASE_ID)
            print('Database with id \'{0}\' created'.format(DATABASE_ID))

        except exceptions.CosmosResourceExistsError:
            db = client.get_database_client(DATABASE_ID)
            print('Database with id \'{0}\' was found'.format(DATABASE_ID))

        # setup container for this sample
        try:
            container = db.create_container(id=CONTAINER_ID, partition_key=PartitionKey(path='/partitionKey'))
            print('Container with id \'{0}\' created'.format(CONTAINER_ID))

        except exceptions.CosmosResourceExistsError:
            container = db.get_container_client(CONTAINER_ID)
            print('Container with id \'{0}\' was found'.format(CONTAINER_ID))

    except exceptions.CosmosHttpResponseError as e:
        print('\nrun_sample has caught an error. {0}'.format(e.message))

    finally:
            print("\nrun_sample done")
    

In [ ]:
create_items(container)

In [ ]:
read_item(container, 'ip1', 'Account1')
read_item(container, 'ip2', 'Account1')
read_items(container)

In [ ]:
query_items(container, 'Account1')

In [ ]:
headers = {'X-Key':''}
data = requests.get("https://api.binaryedge.io/v2/query/search/stats?query=ip:104.18.37.228&type=ports",headers=headers)
print(data.json())

In [ ]:
db.create_container(id="104.18.37.228", partition_key=PartitionKey(path='/partitionKey'))

In [ ]:
dataS = {'id' : "ports1",
            'partitionKey' : 'Account1',
            'date' : datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'portsDescription' : data.json()
            }

In [ ]:
print(dataS)

In [ ]:
container.create_item(body=dataS)